# Using Vertex AI for online serving with NVIDIA Triton

- Demonstrate serving of ensemble models - NVTabular preprocessing + HugeCTR recommender



In [6]:
import nvtabular as nvt

print(nvtabular.__version__)

0.6.1


### Export ensemble

In [10]:
WORKFLOW_PATH = '/home/jupyter/data/criteo_processed_parquet_0.6_t1/workflow'
MODEL_GRAPH_PATH = '/home/jupyter/data/model_t1/graph/deepfm.json'
MODEL_PARAMS_PATH = '/home/jupyter/data/model_t1/parameters/'
OUTPUT_PATH = '/home/jupyter/data/model_ensemble_t1/'
NUM_SLOTS = 26
MAX_NNZ = 2
EMBEDDING_VECTOR_SIZE = 11
NUM_OUTPUTS = 1
ENSEMBLE_NAME = 'deepfm'

In [11]:
CATEGORICAL_COLUMNS=["C" + str(x) for x in range(1, 27)]
CONTINUOUS_COLUMNS=["I" + str(x) for x in range(1, 14)]
LABEL_COLUMNS = ['label']
MAX_BATCH_SIZE = 64

In [12]:
workflow = nvt.Workflow.load(WORKFLOW_PATH)

/opt/conda/lib/python3.7/site-packages/nvtabular/workflow.py:262: UserWarning: Loading workflow generated with python version 3.8.10 (default, Jun  2 2021, 10:49:15) 
[GCC 9.4.0] - but we are running python 3.7.10 | packaged by conda-forge | (default, Sep 13 2021, 19:43:44) 
[GCC 9.4.0]. This might cause issues
  f"Loading workflow generated with {name} version {stored} "
/opt/conda/lib/python3.7/site-packages/nvtabular/workflow.py:277: UserWarning: Loading workflow generated on GPU
  warnings.warn(f"Loading workflow generated on {expected}")


ValueError: unsupported pickle protocol: 5